# Margot's paper "Symmetric ILP: Coloring and small integers": flosn instances
Margot presents 3 flower-snark instances that have cyclic symmetries.

The instance he solves is a 3-edge colouring problem on this class of graphs.

https://doi.org/10.1016/j.disopt.2006.10.008

Since the chromatic index of this problem is 4 (i.e., at least 4 colours are required to colour the edges), all these problems will be infeasible.

https://en.wikipedia.org/wiki/Flower_snark

In [ ]:
import networkx as nx
import pyscipopt as ps

In [ ]:
def generate_flower_snark(n):
    nodes = list(range(4 * n))
    A = nodes[0 * n : 1 * n]
    B = nodes[1 * n : 2 * n]
    C = nodes[2 * n : 3 * n]
    D = nodes[3 * n : 4 * n]

    g : nx.Graph = nx.Graph()
    g.add_nodes_from(nodes)

    # Build n copies of the star graph on 4 vertices, with center A, and surrounding edges B, C, D.
    for i in range(n):
        g.add_edges_from([
            (A[i], B[i]),
            (A[i], C[i]),
            (A[i], D[i])
        ])

    # Construct the n-cycle on the nodes (B1,...,Bn).
    g.add_edges_from([
        (B[i - 1], B[i])
        for i in range(n)
    ])

    # Construct the 2n cycle (C1,...,Cn,D1,...,Dn).
    for i in range(n):
        if i == 0:
            g.add_edge(D[-1], C[0])
            g.add_edge(C[-1], D[0])
        else:
            g.add_edge(C[i - 1], C[i])
            g.add_edge(D[i - 1], D[i])
    
    return g

In [ ]:
def make_edge_colouring(g, c, outputpath):
    colours = range(c)
    model = ps.Model()

    x = {}
    for i, j in sorted(g.edges):
        for c in colours:
            x[tuple(sorted((i, j))), c] = model.addVar(name=f"x[{i},{j},{c}]", vtype="B")

    for i in sorted(g.nodes):
        for c in colours:
            model.addCons(ps.quicksum(x[tuple(sorted((i, j))), c] for j in g.neighbors(i)) <= 1 )

    for i, j in sorted(g.edges):
        model.addCons(ps.quicksum(x[tuple(sorted((i, j))), c] for c in colours) == 1 )

    # model.setObjective(ps.quicksum(x.values()), sense="maximize")

    model.writeProblem(outputpath)

    model.freeProb()
    del model

In [ ]:
c = 3
for n in range(2, 120):
    g = generate_flower_snark(n)
    make_edge_colouring(g, c, f"../instances/flowersnark/flowersnark{n}_{c}.cip")